In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
np.random.seed(9)

In [2]:
# Connect to SQLite file
conn = sqlite3.connect('data/database.sqlite')
cur = conn.cursor()

In [3]:
# create connection and check table names
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")

table_names = cur.fetchall()
table_names

[('Scorecard',)]

In [4]:
# examine columns

cur.execute("""SELECT sql FROM sqlite_master WHERE type = 'table' AND name = 'Scorecard';""")
scorecard_columns = cur.fetchone()

In [5]:
# Create a dataframe of currently operating schools that exclude
# grad program only schools between the year 2002 and 2012
df = pd.read_sql("""
SELECT * 
FROM Scorecard 
WHERE year BETWEEN 2007 AND 2012
AND CURROPER = "Currently certified as operating"
AND PREDDEG != "Entirely graduate-degree granting"
AND PREDDEG != "Not classified"
;
""", conn, index_col='Id')

In [6]:
# Confirm SQL query

df['Year'].unique()

array([2007, 2008, 2009, 2010, 2011, 2012], dtype=int64)

In [7]:
df.shape

(34625, 1730)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34625 entries, 73000 to 116507
Columns: 1730 entries, UNITID to Year
dtypes: float64(191), int64(193), object(1346)
memory usage: 457.3+ MB


In [9]:
df.head(5)

UNITID    OPEID  opeid6                               INSTNM  \
Id                                                                    
73000  100654   100200    1002             Alabama A & M University   
73001  100663   105200    1052  University of Alabama at Birmingham   
73002  100690  2503400   25034                   Amridge University   
73003  100706   105500    1055  University of Alabama in Huntsville   
73004  100724   100500    1005             Alabama State University   

             CITY STABBR         ZIP AccredAgency INSTURL NPCURL  sch_deg  \
Id                                                                          
73000      Normal     AL       35762         None    None   None      3.0   
73001  Birmingham     AL  35294-0110         None    None   None      3.0   
73002  Montgomery     AL  36117-3553         None    None   None      3.0   
73003  Huntsville     AL       35899         None    None   None      3.0   
73004  Montgomery     AL  36101-0271         None    None   None      3.0   

       HCM2         main  NUMBRANCH                                   PREDDEG  \
Id                                                                              
73000     0  Main campus          1  Predominantly bachelor's-degree granting   
73001     0  Main campus          1  Predominantly bachelor's-degree granting   
73002     0  Main campus          1  Predominantly bachelor's-degree granting   
73003     0  Main campus          1  Predominantly bachelor's-degree granting   
73004     0  Main campus          1  Predominantly bachelor's-degree granting   

               HIGHDEG            CONTROL  st_fips  \
Id                                                   
73000  Graduate degree             Public  Alabama   
73001  Graduate degree             Public  Alabama   
73002  Graduate degree  Private nonprofit  Alabama   
73003  Graduate degree             Public  Alabama   
73004  Graduate degree             Public  Alabama   

                                                  region LOCALE locale2  \
Id                                                                        
73000  Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...   None    None   
73001  Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...   None    None   
73002  Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...   None    None   
73003  Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...   None    None   
73004  Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...   None    None   

      LATITUDE LONGITUDE CCBASIC CCUGPROF CCSIZSET  HBCU   PBI ANNHI TRIBAL  \
Id                                                                            
73000     None      None    None     None     None  None  None  None   None   
73001     None      None    None     None     None  None  None  None   None   
73002     None      None    None     None     None  None  None  None   None   
73003     None      None    None     None     None  None  None  None   None   
73004     None      None    None     None     None  None  None  None   None   

      AANAPII   HSI NANTI MENONLY WOMENONLY RELAFFIL  ADM_RATE  ADM_RATE_ALL  \
Id                                                                             
73000    None  None  None    None      None     None    0.3182        0.3182   
73001    None  None  None    None      None     None    0.8778        0.8778   
73002    None  None  None    None      None     None       NaN           NaN   
73003    None  None  None    None      None     None    0.8800        0.8800   
73004    None  None  None    None      None     None    0.4159        0.4159   

       SATVR25  SATVR75  SATMT25  SATMT75  SATWR25  SATWR75  SATVRMID  \
Id                                                                      
73000      NaN      NaN      NaN      NaN      NaN      NaN       NaN   
73001    470.0    620.0    500.0    650.0      NaN      NaN     545.0   
73002      NaN      NaN      NaN      NaN      NaN      NaN       NaN   
73003    490.0  

### NaN's in Y variable 
If NaN's under 0.05% then will drop


In [10]:
X_NaN = df['COMP_ORIG_YR4_RT'].isna().sum()
X_len = len(df['COMP_ORIG_YR4_RT'])
X_NaN, X_len

(1115, 34625)

In [11]:
X_NaN / X_len, X_len-X_NaN

(0.03220216606498195, 33510)

In [12]:
# percent is low enough so will drop
processed_df = df.dropna(subset=['COMP_ORIG_YR4_RT']).copy()
processed_df.shape

(33510, 1730)

# EDA

### Have to deal with NAN's &  'PrivacySuppressed' in columns

In [13]:
# look at columns w NaN's
processed_df.isna().sum()

UNITID                                  0
OPEID                                   0
opeid6                                  0
INSTNM                                  0
CITY                                    0
STABBR                                  0
ZIP                                     0
AccredAgency                        33510
INSTURL                             33510
NPCURL                              33510
sch_deg                                 0
HCM2                                    0
main                                    0
NUMBRANCH                               0
PREDDEG                                 0
HIGHDEG                                 0
CONTROL                                 0
st_fips                                 0
region                                  0
LOCALE                              33510
locale2                             33510
LATITUDE                            33510
LONGITUDE                           33510
CCBASIC                           

In [14]:
# remove columns with all NaN's
perc = 100.0 # Like N %
min_count =  int(((100-perc)/100)*df.shape[0] + 1)
processed_df = processed_df.dropna( axis=1, thresh=min_count)

In [15]:
# Check dataset size after dropping 100% NaN columns
processed_df.shape

(33510, 1626)

In [17]:
# Inspect other columns with NaN's
processed_df.isna().sum()

UNITID                                  0
OPEID                                   0
opeid6                                  0
INSTNM                                  0
CITY                                    0
STABBR                                  0
ZIP                                     0
sch_deg                                 0
HCM2                                    0
main                                    0
NUMBRANCH                               0
PREDDEG                                 0
HIGHDEG                                 0
CONTROL                                 0
st_fips                                 0
region                                  0
ADM_RATE                            19908
ADM_RATE_ALL                        19119
SATVR25                             26007
SATVR75                             26007
SATMT25                             25916
SATMT75                             25916
SATWR25                             29305
SATWR75                           

In [18]:
# creating list of general college info
details = ['UNITID', 'OPEID', 'opeid6', 'INSTNM', 'CITY', 'STABBR', 'ZIP', 'sch_deg', 'HCM2', 'main', 'NUMBRANCH', 'PREDDEG', 'HIGHDEG', 'CONTROL', 'st_fips', 'region', 'Year']

In [19]:
# list of columns to drop if not necessary to project
irrelevant_columns = ['UGDS_AIANOld', # old
                     'UGDS_HISPOld', # old
                     ]

In [20]:
features = ['PCIP01', 'PCIP03', 'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10', 'PCIP11', 'PCIP12', 'PCIP13', 'PCIP14', 'PCIP15', 'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23', 'PCIP24', 'PCIP25', 'PCIP26', 'PCIP27', 'PCIP29', 'PCIP30', 'PCIP31', 'PCIP38', 'PCIP39', 'PCIP40', 'PCIP41', 'PCIP42', 'PCIP43', 'PCIP44', 'PCIP45', 'PCIP46', 'PCIP47', 'PCIP48', 'PCIP49', 'PCIP50', 'PCIP51', 'PCIP52', 'PCIP54']

### Y Variable -  'PrivacySuppressed'
There is a high count of 'privacysuppressed' in the y variable. Upon investigation, all the colleges are beauty, hair, etc schools that is not relevant to the project so they are dropped.

In [21]:
# large count of 'PrivacySuppressed' in Y variable
processed_df['COMP_ORIG_YR4_RT'].value_counts()

PrivacySuppressed      4388
0.0                    1126
0.5                     134
0.319006249047          119
0.343498817967          108
0.312364425163           98
0.554599947876           84
0.666666666667           80
0.606361007848           76
0.710502283105           54
0.188029544485           52
0.201995114963           47
0.6                      45
0.227173905346           45
0.227569840543           42
0.75                     42
0.667844522968           38
1.0                      37
0.625                    32
0.4                      30
0.353503184713           29
0.18609143593            28
0.428571428571           28
0.545454545455           27
0.155505107832           27
0.636363636364           27
0.714285714286           27
0.595174262735           26
0.367829021372           26
0.304494227598           25
0.571428571429           24
0.555555555556           24
0.151714563376           23
0.538461538462           23
0.466666666667           22
0.12670884235       

In [22]:
# creating new dataframe and looking at the value count of colleges
# that have 'privacysuppressed' in the y variable
explore = processed_df[processed_df['COMP_ORIG_YR4_RT']=='PrivacySuppressed']
explore['INSTNM'].value_counts()

Academy of Hair Design                                                         22
Cannella School of Hair Design                                                 12
Moler Barber College                                                           10
J & J Hair Design College                                                       8
Modern Hairstyling Institute                                                    8
North Bennet Street School                                                      6
Sierra College                                                                  6
Delta Beauty College                                                            6
Professional Cosmetology Education Center                                       6
West Virginia University Hospital Departments of Rad Tech and Nutrition         6
Advance Beauty College                                                          6
Heritage Bible College                                                          6
Urban College of

In [23]:
# dropping colleges with 'privacysuppressed'
processed_df = processed_df[~df['COMP_ORIG_YR4_RT'].isin(['PrivacySuppressed'])]

<ipython-input-23-4620f5188428>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  processed_df = processed_df[~df['COMP_ORIG_YR4_RT'].isin(['PrivacySuppressed'])]


In [24]:
#  checking shape of dataframe
processed_df.shape

(29122, 1626)

# Correlation

In [28]:
#  looking at correlations


In [29]:
# corr= df.corr()
# fig, axes = plt.subplots( figsize=(16,15))

# sns.heatmap(data=corr, mask=np.triu(np.ones_like(corr, dtype = bool)), annot=True)

# Models

In [33]:
X = processed_df[features]
y = processed_df['COMP_ORIG_YR4_RT']
X.shape, y.shape

((29122, 38), (29122,))

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Baseline Model

In [35]:
dr = DummyRegressor(strategy='median')
dr.fit(X_train, y_train)
dr.predict(X_train)
dr.score(X_train, y_train)

-1.778686402986729e-05

### First Model

In [36]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

In [37]:
linreg.score(X_train, y_train)

0.3723434434351982